In [1]:
import pandas as pd
from slugify import slugify

from pendulum import now

In [2]:
dataset_name = 'obscana_etanol'
raiz = '../database'

### Bronze

In [3]:
ano_inicio = now().year - 2
ano_fim = now().year - 1

url = f'https://observatoriodacana.com.br/historico-de-producao-e-moagem.php?idMn=31&tipoHistorico=2&acao=visualizar&idTabela=2492&produto=etanol_total&safraIni=1999%2F2000&safraFim={ano_inicio}%2F{ano_fim}&estado=RS%2CSC%2CPR%2CSP%2CRJ%2CMG%2CES%2CMS%2CMT%2CGO%2CDF%2CBA%2CSE%2CAL%2CPE%2CPB%2CRN%2CCE%2CPI%2CMA%2CTO%2CPA%2CAP%2CRO%2CAM%2CAC%2CRR'

(pd.read_html(f'{url}', thousands = ".", decimal = ",")[0]
 .to_html(f'{raiz}/bronze_{dataset_name}.html', index=False)
 )

### Prata

In [4]:
anos = (pd.read_html(f'{raiz}/bronze_{dataset_name}.html', encoding='utf-8')[0]
 .set_index('Estado')
 .rename(columns=lambda col: slugify(col[-4:]))
 .columns
 .tolist()
        )

In [5]:
(pd.read_html(f'{raiz}/bronze_{dataset_name}.html',
                   encoding='utf-8',
                   na_values=0
                   )[0]
      .query('not Estado.str.contains("Região.*|Brasil", case=False)')    
      .rename(columns=lambda col: col if col == 'Estado' else slugify(col[-4:]))
      .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet')
              .filter(['UF-id', 'UF-nome'])
              .drop_duplicates(),
              how='left',
              left_on='Estado',
              right_on='UF-nome'
              )
      .melt(id_vars='UF-id',
            value_vars=anos,
            var_name='ano',
            value_name='safra_m3')
      .assign(**{
            'fonte': 'Fertilizante orgânico vinhaça',
            'ano': lambda df: df['ano'].astype(int)
            })
      .dropna(subset='safra_m3')
      .convert_dtypes()
      .rename(columns={'UF-id':'uf'})
      .sort_values(by=['uf', 'ano'])
      .set_index(['uf', 'ano', 'fonte'])
      .to_parquet(f'{raiz}/prata_{dataset_name}.parquet')
      )

### Ouro

Produção de Etanol ao todo por Estado (1000 m³)

Parâmetros
Teor (kg N/m³)

Cálculo: ((produção estadual * producao)*(parametro_X))*(1000/1000)

Brasil = mil t

NPK na forma do fertilizante orgânico vinhaça incorporado ao solo (t)

In [6]:
producao = 11.5

parametro_N = 0.433
parametro_P = 0.034
parametro_K = 2.206

In [7]:
df = (pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')
      .assign(**{
          'N': lambda df: ((df['safra_m3']*producao)*parametro_N)*(1000/1000),
          'P': lambda df: ((df['safra_m3']*producao)*parametro_P)*(1000/1000),
          'K': lambda df: ((df['safra_m3']*producao)*parametro_K)*(1000/1000)
          })
      .reset_index()
      .melt(id_vars=['uf', 'ano', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .convert_dtypes()
      .set_index(['uf', 'ano', 'fonte'])
)

df.to_csv(f'{raiz}/ouro_{dataset_name}.csv')

In [8]:
df

elemento       valor
uf ano  fonte                                             
11 2009 Fertilizante orgânico vinhaça        N     34.8565
   2010 Fertilizante orgânico vinhaça        N     44.8155
   2011 Fertilizante orgânico vinhaça        N     54.7745
   2012 Fertilizante orgânico vinhaça        N      59.754
   2013 Fertilizante orgânico vinhaça        N     44.8155
...                                        ...         ...
52 2017 Fertilizante orgânico vinhaça        K  111217.696
   2018 Fertilizante orgânico vinhaça        K  117154.042
   2019 Fertilizante orgânico vinhaça        K  124105.148
   2020 Fertilizante orgânico vinhaça        K  140366.677
   2021 Fertilizante orgânico vinhaça        K  132958.929

[1389 rows x 2 columns]